In [ ]:
!git clone https://github.com/lorenzoorsingher/resunet.git
!pip install lpips
!pip install python-dotenv
%cd resunet

In [6]:
import os
import cv2 as cv
from dotenv import load_dotenv

"""
When running this notebook on colab set the COLAB
variable to True
"""
COLAB = False

if COLAB:
    from google.colab.patches import cv2_imshow
    from google.colab import drive
    drive.mount('/content/drive')
    cv.imshow = lambda n,i : cv2_imshow(i)
    !unzip {zip_path} -d {datasetpath}
    zip_path = "/content/drive/MyDrive/tesi/data/lfw/data/images.zip"
    jsonpath = "data/"
    datasetpath = "data/"
    
    colorpath = datasetpath + "color/"
    bwpath = datasetpath + "bw/"
    savepath = "data/checkpoints/"

    

In [ ]:
#@title Dataset standardization

"""
Run this cell if you don't have the json file with the standardization data
for the dataset
"""

from normalize import DataNormalizer

dn = DataNormalizer(colorpath, bwpath, jsonpath)
dn.get_json()

In [4]:
!python train.py -h

usage: train.py [-h] [--json JSON] [--data DATA] [--save SAVE]
                [--insize INSIZE] [--outsize OUTSIZE] [--batch BATCH]
                [--epochs EPOCHS] [--video VIDEO] [--no-video]
                [--load-chkp LOADCHKP]

options:
  -h, --help            show this help message and exit
  --json JSON           Path where json file will be saved/read
  --data DATA           Path to the dataset
  --save SAVE           Path to where checkpoints will be saved
  --insize INSIZE       Size of input image
  --outsize OUTSIZE     Size of output image
  --batch BATCH         Size of batches
  --epochs EPOCHS       Number of epochs
  --video VIDEO         Number of batches between every visualization
  --no-video            Set for no visual debug
  --load-chkp LOADCHKP  Path to checkpoints to be loaded


In [7]:
if COLAB:
    !python train.py --json {jsonpath} --data {datasetpath} -- save{savepath}
else:

    !python train.py 

[INFO] training using cpu...
/home/lollo/miniconda3/envs/envresunet/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lollo/miniconda3/envs/envresunet/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
15114158  total params
ResNetUNet(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPoo